## Develop Kubeflow Pipeline in Notebook

This notebook shows how to build, compile and run a Kubeflow pipeline using Kubeflow SDK in Jupyter notebooks.

This notebook will make sure you understand 
- Develop a Kubeflow pipeline with the KFP SDK
- Submit Kubeflow pipelines using the KFP SDK
- Monitor Kubeflow pipeline running status

For KFP SDK details, please check documentaion [SDK Overview](https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/)

## Prerequisites: Install the pipelines SDK


In [ ]:
!pip install kfp --upgrade --user

## Import kfp sdk

In [ ]:
import kfp
import kfp.dsl as dsl
from kfp import compiler

## Develop your Kubeflow Pipeline 

### Create pipeline component from a python function

In [ ]:
def add_op(a, b):
    
    return dsl.ContainerOp(
        name='Add Operation',
        image='python:3.6.5',
        command=['sh', '-c'],
        arguments=['python -c "print($0+$1)" | tee $2', str(a), str(b), '/tmp/output'],
        file_outputs={
            'data': '/tmp/output',
        }
    )

### Build a pipeline using the component

In [ ]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a=0,
   b=7,
   c=17,
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

### Compile and run the pipeline

Create an Experiment in the Pipeline System

In [ ]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

### Create an Experiment in the Pipeline System

Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

You can also use `default` Experiment to host your pipeline

In [ ]:
# Get or create an experiment and submit a pipeline run
EXPERIMENT_NAME='add_operation'
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

### Submit the pipeline for execution

In [ ]:
# Specify pipeline argument values
arguments = {'a': 7, 'b': 8}

# Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

# This link leads to the run information page. 
# Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working